In [ ]:
#!/usr/bin/env python3

from ipywidgets import VBox, HBox, Button, Output, Layout, Image, HTML
from IPython.display import display, HTML as DHTML
from html import escape
import base64
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

# --- Define projects and images ---
projects = {
    "TOXICITY PREDIITION": {
        "desc": (
            "Toxicity Prediction for chemical compounds using a Feed Forward Neural Network. "
            "This project aims to analyze and predict molecular toxicity based on their chemical properties using anonymized data. "
            "This analysis is part of an AI training session and is conducted in collaboration with "
            "<a href='https://echa.europa.eu' "
            "target='_blank' style='color:#1E90FF; text-decoration:underline;'>"
            "ECHA</a>. "
        ),
        "image": "project_image1.png"
    },
    "PUBLIC WATER USAGE": {
        "desc": "Description for project 2.",
        "image": "project_image2.png"
    },
    "SUSTAINABLE HOUSING": {
        "desc": "Description for project 3.",
        "image": "project_image3.png"
    },
    "INCOME AND EXPENSE": {
        "desc": "Description for project 4.",
        "image": "project_image4.png"
    }
}

# --- Thumbnail settings ---
thumbnail_width = "200px"
thumbnail_height = "150px"

# --- Helper: convert image to base64 ---
def image_to_base64(path):
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

# --- Personal info (photo + description) ---
photo_path = "my_photo.png"  # Replace with your photo filename
photo_b64 = image_to_base64(photo_path)
photo_widget = Image(
    value=base64.b64decode(photo_b64),
    format='png',
    layout=Layout(width='200px', height='auto', border_radius='0px')
)

# --- Use HTML widgets instead of Label ---
name_label = HTML(
    value="<div style='"
          "font-weight:bold; "
          "font-size:18px; "
          "color: rgba(0,0,128,1); "
          "padding:5px 0 5px 30px; "
          "line-height:1.4; "
          "letter-spacing: 8px;"
          "'>"
          "JOHN<span style='margin-left:20px;'>DOE</span>"
          "</div>"
)

description_label = HTML(
    value="<div style='font-size:14px; padding:10px 0 0 0; line-height:1.5;'>"
          "Welcome to my portfolio. Here we will describe using a piece of text whats going on in our portfolio. "
          "Maybe adress the user in a friendly maner and guide them how to use the thumbnails and such</div>"
)

top_row = HBox([photo_widget, name_label], layout=Layout(align_items='center'))
personal_info = VBox([top_row, description_label],
                     layout=Layout(padding='10px', margin='0px 0px 50px 0px'))

# --- Output area for project details ---
details = Output()

# --- Prepare a consistent Pygments formatter style ---
formatter = HtmlFormatter(style='friendly', noclasses=True)

# --- Create thumbnail buttons ---
buttons = []

for name, info in projects.items():
    img_b64 = image_to_base64(info["image"])
    image_uri = f"data:image/png;base64,{img_b64}"

    b = Button(description="", layout=Layout(width=thumbnail_width, height=thumbnail_height, margin="8px"),
               style=dict(button_color="transparent"), tooltip=name)

    css_class = f"thumb-{name.replace(' ', '_')}"
    custom_css = f"""
    <style>
    .{css_class} {{
        background-image: url('{image_uri}');
        background-size: cover;
        background-position: center;
        position: relative;
        font-weight: bold;
        color: white;
        overflow: hidden;
    }}
    .{css_class}::after {{
        content: '{name}';
        display: flex;
        align-items: center;
        justify-content: center;
        width: 100%;
        height: 60px;
        background-color: rgba(0,0,128,0.3);
        position: absolute;
        bottom: 0;
        left: 0;
        font-size: 20px;
        font-family: Arial Black, sans-serif;
        color: white;
        -webkit-text-stroke: 0.5px grey;
        text-align: center;
        white-space: normal;
        line-height: 1.2em;
    }}
    .{css_class}:hover {{
        transform: scale(1.05);
        box-shadow: 0 0 10px #888;
        cursor: pointer;
        transition: transform 0.2s, box-shadow 0.2s;
    }}
    </style>
    """
    display(DHTML(custom_css))
    b.add_class(css_class)
    buttons.append(b)

    def on_click(b, name=name):
        details.clear_output(wait=True)
        with details:
            info = projects[name]

            if name == "TOXICITY PREDIITION":
                code_blocks = [
                    {
                        "guidance": "Install the required packages for chemistry and data handling",
                        "code": """
# --- Install necessary packages ---
pip install rdkit
pip install tqdm
"""
                    },
                    {
                        "guidance": "Verify that Python and package versions are compatible",
                        "code": """
# --- Display Python and required package versions to check if versions are compatible ---
import sys
from packaging.version import Version, InvalidVersion

# --- Define a function to get package versions ---
def get_packages(pkgs):
    versions = []
    for p in pkgs:
        try:
            imported = __import__(p)
            try:
                versions.append(imported.__version__)
            except AttributeError:
                try:
                    versions.append(imported.version)
                except AttributeError:
                    try:
                        versions.append(imported.version_info)
                    except AttributeError:
                        versions.append('0.0')
        except ImportError:
            print(f'[FAIL]: {p} is not installed and/or cannot be imported.')
            versions.append('N/A')
    return versions

# --- Print Python versions ---
print(f"python: {sys.version}")

# --- Print package versions ---
pkgs = ['torch', 'numpy', 'pandas', 'sklearn', 'rdkit', 'matplotlib', 'seaborn']
version = get_packages(pkgs)
for pkg, version in zip(pkgs, version):
    print(f"{pkg}: {version}")
"""
                    }
                ]

                rendered_cells = []
                for block in code_blocks:
                    highlighted_code = highlight(block["code"].strip(), PythonLexer(), formatter)

                    # force transparent inner background + same color everywhere
                    wrapper_style = """
                    <style>
                    .code-wrapper pre,
                    .code-wrapper code,
                    .code-wrapper span {
                        background: rgb(245, 245, 245) !important;
                        background-color: rgb(245, 245, 245) !important;
                        border: none !important;
                        box-shadow: none !important;
                    }
                    .code-wrapper pre {
                        margin: 0 !important;
                        padding: 0 !important;
                    }
                    </style>
                    """

                    wrapper_div = f"""
                    {wrapper_style}
                    <div class="code-wrapper" style="
                        border-radius: 10px;
                        overflow-x: auto;
                        background-color: rgb(245, 245, 245);
                        box-shadow: inset 0 0 4px rgba(0,0,0,0.0);
                        padding: 30px;">
                        {highlighted_code}
                    </div>
                    """

                    rendered_cells.append(f"""
                        <div style="margin-top: 20px;">
                            <div style="font-size:15px; font-weight:bold; color:#003366; margin-bottom:6px;">
                                {escape(block['guidance'])}
                            </div>
                            {wrapper_div}
                        </div>
                    """)

                code_html = "\n".join(rendered_cells)
            else:
                code_html = "<p><em>More details coming soon...</em></p>"

            html_content = f"""
            <div style="background-color: rgba(0,0,128,0.7);
                        color:white;
                        font-family: Arial Black, sans-serif;
                        font-size: 16px;
                        padding:20px;
                        border-radius:10px;">
                <h3 style="margin:0;">{escape(name)}</h3>
            </div>

            <p style='color:#5A5A5A; padding:20px 0'>{info['desc']}</p>
            <hr>
            {code_html}
            """
            display(DHTML(html_content))

    b.on_click(on_click)

grid = VBox([
    HBox(buttons[:2], layout=Layout(justify_content="flex-start", gap='40px')),
    HBox(buttons[2:], layout=Layout(justify_content="flex-start", gap='40px'))
], layout=Layout(width="100%", padding="0px", margin="0px", gap='10px'))

left_column = VBox([personal_info, grid], layout=Layout(width="35%", padding="50px", margin="0px"))

#right_column = VBox([details], layout=Layout(width="85%", border="0px solid gray", padding="0px"))
# Right column becomes scrollable independently
right_column = VBox([details],
                    layout=Layout(width="65%", padding="0px", margin="0px",
                                  border="0px solid gray",
                                  overflow_y='auto',
                                  height='900px'))  # fixed height or adjust

ui = HBox([left_column, right_column], layout=Layout(width="100%", padding="0px", margin="0px"))
display(ui)
